# Stochastic Framework Through Year Calendar

In [2]:
import pandas as pd
import numpy as np

In [3]:
schedules = pd.read_csv("data/Schedules_1979-2023.csv")
schedules.head()
    

,Date,Year,HomeTm,AwayTm,Home,Away
0,1978-10-13,1979,DEN,SAS,Denver Nuggets,San Antonio Spurs
1,1978-10-13,1979,DET,NJN,Detroit Pistons,New Jersey Nets
2,1978-10-13,1979,IND,ATL,Indiana Pacers,Atlanta Hawks
3,1978-10-13,1979,NYK,HOU,New York Knicks,Houston Rockets
4,1978-10-13,1979,PHI,LAL,Philadelphia 76ers,Los Angeles Lakers


In [4]:
def run_through_calendar(year, gamma, model_pred):

     schedules_year = schedules.copy()[schedules.Year==year].reset_index(drop=True)
     model_pred_year = model_pred.copy()[model_pred.Year == year].reset_index(drop=True)
     # Take list of teams as home and away team and initiate the keys of pred_dic
     Tm = pd.unique(schedules_year[['HomeTm', 'AwayTm']].values.ravel('K'))
     pred_dic = {value: 0 for value in Tm}

     for k in range(len(schedules_year)):
          home_pred = model_pred_year[model_pred_year.Tm == schedules_year.loc[k, "HomeTm"]]['Pred'].item()
          away_pred = model_pred_year[model_pred_year.Tm == schedules_year.loc[k, "AwayTm"]]['Pred'].item()
          
          if home_pred + gamma > away_pred:
               pred_dic[schedules_year.loc[k, "HomeTm"]] +=1
          
          elif home_pred + gamma < away_pred:
               pred_dic[schedules_year.loc[k, "AwayTm"]] +=1
          
          else: # Rare case of equality: then just random
               outcome = np.random.randint(2)
               pred_dic[schedules_year.loc[k, "HomeTm"]] += outcome
               pred_dic[schedules_year.loc[k, "AwayTm"]] += 1-outcome
     
     return pred_dic


In [5]:
pred_LR = pd.read_csv('data/pred_test_LR.csv')
pred_LR.head()

,Year,Tm,Real,Pred
0,2014,BOS,25.0,33.722703
1,2014,NYK,37.0,48.635391
2,2014,PHI,19.0,28.445007
3,2014,ATL,38.0,47.303396
4,2014,CLE,33.0,31.400120


In [6]:
pred_dict = run_through_calendar(2018, 1, pred_LR)

In [7]:
pred_dict

{'CLE': 73,
 'GSW': 82,
 'DET': 32,
 'IND': 25,
 'ORL': 6,
 'WAS': 54,
 'BOS': 63,
 'MEM': 53,
 'DAL': 28,
 'UTA': 44,
 'SAS': 74,
 'PHO': 18,
 'SAC': 5,
 'TOR': 69,
 'OKC': 71,
 'LAL': 6,
 'MIL': 37,
 'PHI': 14,
 'CHO': 42,
 'BRK': 0,
 'MIN': 63,
 'NOP': 42,
 'NYK': 16,
 'MIA': 48,
 'CHI': 25,
 'HOU': 79,
 'DEN': 46,
 'LAC': 64,
 'POR': 37,
 'ATL': 14}

In [8]:
target = pd.read_csv("data/target_1979-2023.csv")

In [173]:
target

,Tm,Year,W/L%
0,WSB,1979,0.659
1,SEA,1979,0.634
2,PHO,1979,0.610
3,SAS,1979,0.585
4,KCK,1979,0.585
...,...,...,...
1241,POR,2023,0.402
1242,CHO,2023,0.329
1243,HOU,2023,0.268
1244,SAS,2023,0.268
